# Pedram ShahSafi

# pd.shahsafi@gmail.com

<img src="pics/1.jpg" />

# object-relational mapper (ORM)
ORM stands for "Object to Relational Mapping" where

The Object part is the one you use with your programming language ( python in this case )

The Relational part is a Relational Database Manager System ( A database that is ) there are other types of databases but the most popular is relational ( you know tables, columns, pk fk etc eg Oracle MySQL, MS-SQL )

And finally the Mapping part is where you do a bridge between your objects and your tables.

# SQLAlchemy
SQLAlchemy is most famous for its object-relational mapper (ORM), an optional component that provides the data mapper pattern, where classes can be mapped to the database in open ended, multiple ways - allowing the object model and database schema to develop in a cleanly decoupled way from the beginning.




# Flask and SQLAlchemy without the Flask-SQLAlchemy Extension
https://dev.to/nestedsoftware/flask-and-sqlalchemy-without-the-flask-sqlalchemy-extension-3cf8

When using SQLAlchemy with Flask, the standard approach is to use the Flask-SQLAlchemy extension.

However, this extension has some issues. In particular, we have to use a base class for our sqlalchemy models that creates a dependency on flask (via flask_sqlalchemy.SQLAlchemy.db.Model). Also, an application may not require the additional functionality that the extension provides, such as pagination support.

Let's see if we can find a way to use plain SQLAlchemy in our Flask applications without relying on this extension.

#### Below is the code that sets up the SQLAlchemy session (db.py):

In [ ]:
import os

from sqlalchemy import create_engine

from sqlalchemy.orm import scoped_session
from sqlalchemy.orm import sessionmaker

engine = create_engine(os.environ['SQLALCHEMY_URL'])

Session = scoped_session(sessionmaker(bind=engine))

The key here is scoped_session: Now when we use Session, SQLAlchemy will check to see if a thread-local session exists. If it already exists, then it will use it, otherwise it will create one first.

#### The following code bootstraps the Flask application (__init__.py):

In [ ]:
from flask import Flask

from .db import Session

from .hello import hello_blueprint

app = Flask(__name__)
app.register_blueprint(hello_blueprint)

@app.teardown_appcontext
def cleanup(resp_or_exc):
    Session.remove()

The @app.teardown_appcontext decorator will cause the supplied callback, cleanup, to be executed when the current application context is torn down. This happens after each request. That way we make sure to release the resources used by a session after each request.

In our Flask application, we can now use Session to interact with our database.
#### For example (hello.py):

In [ ]:
import json

from flask import Blueprint

from .db import Session

from .models import Message

hello_blueprint = Blueprint('hello', __name__)

@hello_blueprint.route('/messages')
def messages():
    values = Session.query(Message).all()

    results = [{ 'message': value.message } for value in values]

    return (json.dumps(results), 200, { 'content_type': 'application/json' })

If we just need session management without the other features that Flask-SQLAlchemy provides, such as pagination, signalling events (SQLAlchemy has its own anyway), driver hacks, SQLAlchemy configuration via Flask, and a few others, then this code should be sufficient for integrating SQLAlchemy into a Flask application.

We also get the benefit of not having to create a dependency on Flask for our SQLAlchemy models. Below we're just using the standard sqlalchemy.ext.declarative.declarative_base <b>(models.py)</b>:

In [ ]:
from sqlalchemy.ext.declarative import declarative_base

from sqlalchemy import Column, Integer, String

Base = declarative_base()

class Message(Base):
    __tablename__ = 'messages'
    id = Column(Integer, primary_key=True)
    message = Column(String)

    def __repr__(self):
        return "<Message(message='%s')>" % (self.message)

# Flask-SQLAlchemy
Flask-SQLAlchemy is an extension for Flask that adds support for SQLAlchemy to your application. It requires SQLAlchemy 0.8 or higher. It aims to simplify using SQLAlchemy with Flask by providing useful defaults and extra helpers that make it easier to accomplish common tasks.

> http://flask-sqlalchemy.pocoo.org

## Declaring Models

> http://flask-sqlalchemy.pocoo.org/2.3/models/

The <b>baseclass</b> for all your models is called <b>db.Model</b>. It’s stored on the SQLAlchemy instance you have to create.
Some parts that are <b>required in SQLAlchemy are optional in Flask-SQLAlchemy</b>. 
For instance the <b>table name is automatically set</b> for you unless overridden. It’s derived from the class name converted to lowercase and with “CamelCase” converted to <b>“camel_case”</b>. To override the table name, set the __tablename__ class attribute.


In [ ]:
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)

    def __repr__(self):
        return '<User %r>' % self.username

Use <b>Column</b> to define a column. The name of the column is the name you assign it to. If you want to use a different name in the table you can provide an optional first argument which is a string with the desired column name.

Primary keys are marked with <b>primary_key=True</b>. Multiple keys can be marked as primary keys in which case they become a <b>compound primary key</b>.

The <b>types of the column are the first argument to Column</b>. You can either provide them directly or call them to further specify them (like providing a length).


1. Integer	an integer
2. String(size)	a string with a maximum length (optional in some databases, e.g. PostgreSQL)
3. Text	some longer unicode text
4. DateTime	date and time expressed as Python datetime object.
5. Float	stores floating point values
6. Boolean	stores a boolean value
7. PickleType	stores a pickled Python object
8. LargeBinary	stores large arbitrary binary data

## One-to-Many Relationships
The most common relationships are one-to-many relationships.<b> Because relationships are declared before they are established you can use strings to refer to classes that are not created yet</b> (for instance if Person defines a relationship to Address which is declared later in the file).

Relationships are expressed with the <b>relationship()</b> function. However the foreign key has to be separately declared with the ForeignKey class:

In [ ]:
class Person(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(50), nullable=False)
    addresses = db.relationship('Address', backref='person', lazy=True)

class Address(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    email = db.Column(db.String(120), nullable=False)
    person_id = db.Column(db.Integer, db.ForeignKey('person.id'), nullable=False)

### What does db.relationship() do? 

That function returns a new property that can do multiple things. In this case we told it to point to the Address class and load multiple of those. How does it know that this will return more than one address? Because SQLAlchemy guesses a useful default from your declaration. If you would want to have a <b> one-to-one relationship you can pass uselist=False to relationship() </b>.

Since a person with no name or an email address with no address associated makes no sense, <b>nullable=False tells SQLAlchemy to create the column as NOT NULL </b>. This is implied for primary key columns, but it’s a good idea to specify it for all other columns to make it clear to other people working on your code that you did actually want a nullable column and did not just forget to add it.

So what do <b>backref and lazy</b> mean? backref is a simple way to also declare a new property on the Address class. You can then also use my_address.person to get to the person at that address.

lazy defines when SQLAlchemy will load the data from the database:

1. 'select' / True (which is the default, but explicit is better than implicit) means that SQLAlchemy will load the data as necessary in one go using a standard select statement.

2. 'joined' / False tells SQLAlchemy to load the relationship in the same query as the parent using a JOIN statement.

3. 'subquery' works like 'joined' but instead SQLAlchemy will use a subquery.

4. 'dynamic' is special and can be useful if you have many items and always want to apply additional SQL filters to them. Instead of loading the items SQLAlchemy will return another query object which you can further refine before loading the items. Note that this cannot be turned into a different loading strategy when querying so it’s often a good idea to avoid using this in favor of lazy=True. A query object equivalent to a dynamic user.addresses relationship can be created using Address.query.with_parent(user) while still being able to use lazy or eager loading on the relationship itself as necessary.

### How do you define the lazy status for backrefs? By using the backref() function:

In [ ]:
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(50), nullable=False)
    addresses = db.relationship('Address', lazy='select',
        backref=db.backref('person', lazy='joined'))

## Many-to-Many Relationships
If you want to use many-to-many relationships you will need to define a helper table that is used for the relationship. For this helper table it is <b>strongly recommended to not use a model but an actual table</b>:

In [ ]:
tags = db.Table('tags',
    db.Column('tag_id', db.Integer, db.ForeignKey('tag.id'), primary_key=True),
    db.Column('page_id', db.Integer, db.ForeignKey('page.id'), primary_key=True)
)

class Page(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    tags = db.relationship('Tag', secondary=tags, lazy='subquery',
        backref=db.backref('pages', lazy=True))

class Tag(db.Model):
    id = db.Column(db.Integer, primary_key=True)

Here we configured Page.tags to be loaded immediately after loading a Page, but using a separate query. This always results in two queries when retrieving a Page, but when querying for multiple pages you will not get additional queries.

The list of pages for a tag on the other hand is something that’s rarely needed. For example, you won’t need that list when retrieving the tags for a specific page. Therefore, the backref is set to be lazy-loaded so that accessing it for the first time will trigger a query to get the list of pages for that tag. If you need to apply further query options on that list, you could either switch to the 'dynamic' strategy - with the drawbacks mentioned above - or get a query object using Page.query.with_parent(some_tag) and then use it exactly as you would with the query object from a dynamic relationship.


## Select, Insert, Delete

### Inserting Records
Before we can query something we will have to insert some data. All your models should have a constructor, so make sure to add one if you forgot. Constructors are only used by you, not by SQLAlchemy internally so it’s entirely up to you how you define them.

### Inserting data into the database is a three step process:

1. Create the Python object
2. Add it to the session
3. Commit the session

The session here is not the Flask session, but the Flask-SQLAlchemy one. It is essentially a beefed up version of a database transaction. This is how it works:

In [ ]:
>>> from yourapp import User
>>> me = User('admin', 'admin@example.com')
>>> db.session.add(me)
>>> db.session.commit()

Alright, that was not hard. What happens at what point? Before you add the object to the session, SQLAlchemy basically does not plan on adding it to the transaction. That is good because you can still discard the changes. For example think about creating the post at a page but you only want to pass the post to the template for preview rendering instead of storing it in the database.

The <b> add() </b> function call then adds the object. It will issue an INSERT statement for the database but because the transaction is still not committed you won’t get an ID back immediately. If you do the commit, your user will have an ID:

In [ ]:
>>> me.id
1

## Deleting Records
Deleting records is very similar, instead of add() use delete():

In [ ]:
>>> db.session.delete(me)
>>> db.session.commit()

## Querying Records
So how do we get data back out of our database? For this purpose Flask-SQLAlchemy provides a query attribute on your Model class. When you access it you will get back a new query object over all records. You can then use methods like <b>filter()</b> to filter the records before you fire the select with <b>all() or first()</b>. If you want to go <b>by primary key</b> you can also use <b>get()</b>.

The following queries assume following entries in the database:




In [ ]:
id	username	email
1	admin	admin@example.com
2	peter	peter@example.org
3	guest	guest@example.com

### Retrieve a user by username:

In [ ]:
>>> peter = User.query.filter_by(username='peter').first()
>>> peter.id
2
>>> peter.email
u'peter@example.org'

Same as above but for a non existing username gives None:

In [ ]:
>>> missing = User.query.filter_by(username='missing').first()
>>> missing is None
True

### Selecting a bunch of users by a more complex expression:

In [ ]:
>>> User.query.filter(User.email.endswith('@example.com')).all()
[<User u'admin'>, <User u'guest'>]

### Ordering users by something:



In [ ]:
>>> User.query.order_by(User.username).all()
[<User u'admin'>, <User u'guest'>, <User u'peter'>]

### Limiting users:

In [ ]:
>>> User.query.limit(1).all()
[<User u'admin'>]

### Getting user by primary key:

In [ ]:
>>> User.query.get(1)
<User u'admin'>
Queries in Views

If you write a Flask view function it’s often very handy to return a <b>404 error for missing entries</b>.

Because this is a very common idiom, Flask-SQLAlchemy provides a helper for this exact purpose. Instead of <b>get() one can use get_or_404()</b> and instead of <b>first() first_or_404()</b>. This will <b>raise 404 errors instead of returning None</b>:

In [ ]:
@app.route('/user/<username>')
def show_user(username):
    user = User.query.filter_by(username=username).first_or_404()
    return render_template('show_user.html', user=user)

## My small web application

we are going to study the ORM techniques of Flask-SQLAlchemy and build a small web application.

> Step 1 − Install Flask-SQLAlchemy extension.

In [ ]:
pip install flask-sqlalchemy

> Step 2 − You need to import SQLAlchemy class from this module.

In [ ]:
from flask_sqlalchemy import SQLAlchemy

> Step 3 − Now create a Flask application object and set URI for the database to be used.

In [ ]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///students.sqlite3'

> Step 4 − Then create an object of SQLAlchemy class with application object as the parameter. This object contains helper functions for ORM operations. It also provides a parent Model class using which user defined models are declared. In the snippet below, a students model is created.

In [ ]:
db = SQLAlchemy(app)

class students(db.Model):
   id = db.Column('student_id', db.Integer, primary_key = True)
   name = db.Column(db.String(100))
   city = db.Column(db.String(50))  
   addr = db.Column(db.String(200))
   pin = db.Column(db.String(10))

    def __init__(self, name, city, addr,pin):
       self.name = name
       self.city = city
       self.addr = addr
       self.pin = pin

> Step 5 − To create / use database mentioned in URI, run the create_all() method.

In [ ]:
db.create_all()

The following session methods perform CRUD operations −

1. db.session.add(model object) − inserts a record into mapped table

2. db.session.delete(model object) − deletes record from table

3. model.query.all() − retrieves all records from table (corresponding to SELECT query).

You can apply a filter to the retrieved record set by using the filter attribute. 

For instance, in order to retrieve records with <b>city = ’Hyderabad’ in students</b> table, use following statement −

In [ ]:
Students.query.filter_by(city = ’Hyderabad’).all()

With this much of background, now we shall provide view functions for our application to add a student data.

The entry point of the application is show_all() function bound to ‘/’ URL. The Record set of students table is sent as parameter to the HTML template. The Server side code in the template renders the records in HTML table form.

In [ ]:
@app.route('/')
def show_all():
   return render_template('show_all.html', students = students.query.all() )

The HTML script of the template <b>(‘show_all.html’)</b> is like this −

In [ ]:
<!DOCTYPE html>
<html lang = "en">
   <head></head>
   <body>
      
      <h3>
         <a href = "{{ url_for('show_all') }}">Comments - Flask 
            SQLAlchemy example</a>
      </h3>
      
      <hr/>
      {%- for message in get_flashed_messages() %}
         {{ message }}
      {%- endfor %}

      <h3>Students (<a href = "{{ url_for('new') }}">Add Student
         </a>)</h3>
      
      <table>
         <thead>
            <tr>
               <th>Name</th>
               <th>City</th>
               <th>Address</th>
               <th>Pin</th>
            </tr>
         </thead>
         
         <tbody>
            {% for student in students %}
               <tr>
                  <td>{{ student.name }}</td>
                  <td>{{ student.city }}</td>
                  <td>{{ student.addr }}</td>
                  <td>{{ student.pin }}</td>
               </tr>
            {% endfor %}
         </tbody>
      </table>
      
   </body>
</html>

The above page contains a hyperlink to ‘/new’ URL mapping new() function. When clicked, it opens a Student Information form.

The data is posted to the same URL in POST method.

#### new.html

In [ ]:
<!DOCTYPE html>
<html>
   <body>
   
      <h3>Students - Flask SQLAlchemy example</h3>
      <hr/>
      
      {%- for category, message in get_flashed_messages(with_categories = true) %}
         <div class = "alert alert-danger">
            {{ message }}
         </div>
      {%- endfor %}
      
      <form action = "{{ request.path }}" method = "post">
         <label for = "name">Name</label><br>
         <input type = "text" name = "name" placeholder = "Name" /><br>
         <label for = "email">City</label><br>
         <input type = "text" name = "city" placeholder = "city" /><br>
         <label for = "addr">addr</label><br>
         <textarea name = "addr" placeholder = "addr"></textarea><br>
         <label for = "PIN">City</label><br>
         <input type = "text" name = "pin" placeholder = "pin" /><br>
         <input type = "submit" value = "Submit" />
      </form>
      
   </body>
</html>

When the http method is detected as POST, the form data is added in the students table and the application returns to homepage showing the added data.

In [ ]:
@app.route('/new', methods = ['GET', 'POST'])
def new():
   if request.method == 'POST':
      if not request.form['name'] or not request.form['city'] or not request.form['addr']:
         flash('Please enter all the fields', 'error')
      else:
         student = students(request.form['name'], request.form['city'],
            request.form['addr'], request.form['pin'])
         
         db.session.add(student)
         db.session.commit()
         
         flash('Record was successfully added')
         return redirect(url_for('show_all'))
   return render_template('new.html')

Given below is the complete code of application <b>(app.py)</b>.

In [ ]:
from flask import Flask, request, flash, url_for, redirect, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///students.sqlite3'
app.config['SECRET_KEY'] = "random string"

db = SQLAlchemy(app)

class students(db.Model):
   id = db.Column('student_id', db.Integer, primary_key = True)
   name = db.Column(db.String(100))
   city = db.Column(db.String(50))
   addr = db.Column(db.String(200)) 
   pin = db.Column(db.String(10))

def __init__(self, name, city, addr,pin):
   self.name = name
   self.city = city
   self.addr = addr
   self.pin = pin

@app.route('/')
def show_all():
   return render_template('show_all.html', students = students.query.all() )

@app.route('/new', methods = ['GET', 'POST'])
def new():
   if request.method == 'POST':
      if not request.form['name'] or not request.form['city'] or not request.form['addr']:
         flash('Please enter all the fields', 'error')
      else:
         student = students(request.form['name'], request.form['city'],
            request.form['addr'], request.form['pin'])
         
         db.session.add(student)
         db.session.commit()
         flash('Record was successfully added')
         return redirect(url_for('show_all'))
   return render_template('new.html')

if __name__ == '__main__':
   db.create_all()
   app.run(debug = True)